# modelo 07 - yolooo + yoloJ

yolo > mask detection > count

- https://chatgpt.com/share/67dc37e8-77b0-8009-93fa-b2e6ce5ee496

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_pickle('Model_Train.pkl')
df_val = pd.read_pickle('Model_Val.pkl')

In [3]:
set(df_train["img_placement"]) | set(df_val["img_placement"])

{'S-unknown', 'openspace', 'r_openspace', 'roof'}

In [4]:
set(df_train["img_origin"]) | set(df_val["img_origin"])

{'D', 'S'}

In [5]:
def hot_encod_img_placement(df):
    extra_features_all = []
    for i, row in df.iterrows():
        extra_features = []
        if row["img_placement"] == "roof":
            extra_features += [1, 0, 0, 0]
        elif row["img_placement"] == "r_openspace":
            extra_features += [0, 1, 0, 0]
        elif row["img_placement"] == "openspace":
            extra_features += [0, 0, 1, 0]
        elif row["img_placement"] == "S-unknown":
            extra_features += [0, 0, 0, 1]
        else:
            extra_features += [0, 0, 0, 0]
        if row["img_origin"] == "D":
            extra_features += [1, 0]
        elif row["img_origin"] == "S":
            extra_features += [0, 1]
        else:
            extra_features += [0, 0] 
        extra_features_all.append(extra_features)
    
    df["more_features"] = extra_features_all

    df.drop(columns=["img_origin"], inplace=True)
    df.drop(columns=["img_placement"], inplace=True)

hot_encod_img_placement(df_train)
hot_encod_img_placement(df_val)

In [6]:
df_train.sample(3)

,img_id,nr_boil,polygons_boil,nr_pan,polygons_pan,more_features
71,IDq4qotpNfMAIxW,[],[],[12],"[[(246.6909090909091, 346.72598584428715), (20...","[1, 0, 0, 0, 1, 0]"
2084,ID0AGL1m6y7,[],[],"[1, 1]","[[(203.52, 285.8666666666667), (211.712, 259.0...","[1, 0, 0, 0, 1, 0]"
1500,IDhMdRj,[],[],[1],"[[(192.768, 295.936), (176.768, 253.6106666666...","[1, 0, 0, 0, 1, 0]"


---

## get ready for yolo

In [7]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

folder = "val"

def polygons_to_segmentation_mask(name, polygons):
    # 512 x 512
    image = np.zeros((512, 512), dtype=np.uint8)
    for polygon in polygons[0]:
        poly = np.array([polygon], dtype=np.int32)
        cv2.fillPoly(image, [poly], 1)
    for polygon in polygons[1]:
        poly = np.array([polygon], dtype=np.int32)
        cv2.fillPoly(image, [poly], 2)

    # save image as name_mask.png
    cv2.imwrite(f"ModelBuild/model08/{folder}/masks/{name}.png", image)

def resize_image(name, size=(512, 512)):
    image_file = f"images/{name}.jpg"
    image = cv2.imread(image_file)
    resized_image = cv2.resize(image, size)
    cv2.imwrite(f"ModelBuild/model08/{folder}/images/{name}.png", resized_image)

"""for i, row in df_val.iterrows():

    if i % 100 == 0:
        print(i, len(df_val))

    polygons_to_segmentation_mask(row["img_id"], [row["polygons_boil"], row["polygons_pan"]])
    resize_image(row["img_id"])"""


'for i, row in df_val.iterrows():\n\n    if i % 100 == 0:\n        print(i, len(df_val))\n\n    polygons_to_segmentation_mask(row["img_id"], [row["polygons_boil"], row["polygons_pan"]])\n    resize_image(row["img_id"])'

In [8]:
from ultralytics.data.converter import convert_segment_masks_to_yolo_seg

# masks to yolo format
folder = "val"

#convert_segment_masks_to_yolo_seg(masks_dir=f"/Users/hugover/Desktop/CAA01/ModelBuild/model08/{folder}/masks",
#                                  output_dir=f"/Users/hugover/Desktop/CAA01/ModelBuild/model08/{folder}/labels", classes=2)



#val_files = os.listdir(f"ModelBuild/model08/{folder}/images")
#for val_file in val_files:
#    if val_file.endswith(".txt"):
#        with open(f"ModelBuild/model08/{folder}/labels/{val_file}", "r") as f:
#            lines = f.readlines()
#            lines = [line.replace("254 ", "0 ") for line in lines]
#        with open(f"ModelBuild/model08/{folder}/labels/{val_file}", "w") as f:
#            f.writelines(lines)


In [9]:
from ultralytics import YOLO


# Load YOLOv8 Model
"""model = YOLO("yolo11m-seg")

# Train the model
model.train(
    data='ModelBuild/model08/data.yaml',  # Path to your data.yaml
    epochs=30,                           # Number of epochs
    imgsz=512,                           # Image size
    batch=32,                            # Batch size
    augment=True,                        # Enable data augmentation
    project='ModelBuild',                # Path where training results will be saved (logs, checkpoints)
    name='model08_yolo',                 # Name of the specific experiment folder (creates a new folder)
    exist_ok=True,                         # Overwrite the folder if it already exists
    patience=5,                          # Stop training if loss does not improve for this number of consecutive epochs
)"""

'model = YOLO("yolo11m-seg")\n\n# Train the model\nmodel.train(\n    data=\'ModelBuild/model08/data.yaml\',  # Path to your data.yaml\n    epochs=30,                           # Number of epochs\n    imgsz=512,                           # Image size\n    batch=32,                            # Batch size\n    augment=True,                        # Enable data augmentation\n    project=\'ModelBuild\',                # Path where training results will be saved (logs, checkpoints)\n    name=\'model08_yolo\',                 # Name of the specific experiment folder (creates a new folder)\n    exist_ok=True,                         # Overwrite the folder if it already exists\n    patience=5,                          # Stop training if loss does not improve for this number of consecutive epochs\n)'

iterrempido na epoch 6 (incluisive)

In [10]:
from ultralytics import YOLO

# Load the best trained model
model = YOLO("ModelBuild/model08_yolo/weights/best.pt")  # Adjust path to your best model
model = YOLO("ModelBuild/model08_yoloJ/weights/best.pt")
model = YOLO("ModelBuild/model08_yoloJ02/weights/best.pt")

# Run validation
#metrics = model.val()

---
---

submissao

In [11]:
test = pd.read_csv("Test.csv")
submission = pd.read_csv("zindi_files/SampleSubmission.csv")

In [12]:
test.head()

,ID,img_origin,placement
0,ID00qprY,D,roof
1,ID01AciUc,D,roof
2,ID0328D,D,roof
3,ID05WxObCFTs9,D,roof
4,ID06AdCmLMlkO,S,S-unknown


In [13]:
submission.head()

,ID,Target
0,ID00qprY_boil,0
1,ID00qprY_pan,0
2,ID01AciUc_boil,0
3,ID01AciUc_pan,0
4,ID0328D_boil,0


In [14]:
def resize_image(name, size=(512, 512)):
    image_file = f"images/{name}.jpg"
    image = cv2.imread(image_file)
    resized_image = cv2.resize(image, size)
    cv2.imwrite(f"ModelBuild/model08_subm/{name}.png", resized_image)

In [15]:
for i, row in test.iterrows():
    if i % 100 == 0:
        print(i, len(test))

    #resize_image(row["ID"])
    results = model(f"ModelBuild/model08_subm/{row['ID']}.png")
    result = results[0]

    # Count occurrences of each detected class
    class_counts = {}

    for cls in result.boxes.cls.tolist():  # Get list of detected class indices
        class_counts[int(cls)] = class_counts.get(int(cls), 0) + 1

    # Map class indices to class names
    class_names = model.names  # Dictionary mapping class index to name
    named_counts = {class_names[k]: v for k, v in class_counts.items()}

    if "boil_alone" in named_counts:
        submission.loc[submission["ID"] == f'{row["ID"]}_boil', "Target"] = named_counts["boil_alone"]
    
    if "pan_alone" in named_counts:
        submission.loc[submission["ID"] == f'{row["ID"]}_pan', "Target"] = named_counts["pan_alone"]

submission.head(10)

0 1107

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model08_subm/ID00qprY.png: 512x512 1 pan_alone, 1069.4ms
Speed: 4.2ms preprocess, 1069.4ms inference, 13.1ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model08_subm/ID01AciUc.png: 512x512 1 pan_alone, 1053.0ms
Speed: 2.8ms preprocess, 1053.0ms inference, 4.6ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model08_subm/ID0328D.png: 512x512 2 pan_alones, 1144.5ms
Speed: 72.3ms preprocess, 1144.5ms inference, 7.9ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model08_subm/ID05WxObCFTs9.png: 512x512 2 pan_alones, 745.1ms
Speed: 3.7ms preprocess, 745.1ms inference, 2.6ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model08_subm/ID06AdCmLMlkO.png: 512x512 1 boil_alone, 503.8ms
Speed: 1.1ms preprocess, 503.8ms inference, 2

,ID,Target
0,ID00qprY_boil,0
1,ID00qprY_pan,1
2,ID01AciUc_boil,0
3,ID01AciUc_pan,1
4,ID0328D_boil,0
5,ID0328D_pan,2
6,ID05WxObCFTs9_boil,0
7,ID05WxObCFTs9_pan,2
8,ID06AdCmLMlkO_boil,1
9,ID06AdCmLMlkO_pan,0


In [16]:
submission.to_csv("zindi_files/model08_yoloJ02_submission.csv", index=False)

**public score:** 2.64 (ModelBuild/model08_yoloJ/weights/best.pt)